In [1]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
import nltk
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib import pyplot as plt
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, accuracy_score,hamming_loss
import numpy as np
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
import sklearn.utils

[nltk_data] Downloading package punkt to C:\Users\zxc
[nltk_data]     ghoul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
db_name = 'recipe.db'
conn = sqlite3.connect(db_name)

table_name = 'recipe'

#забираем доступные категории
categories_sql = f"select c.id from category c " \
                 f"join recipe_categories rc on " \
                 f"c.id = rc.cat_id " \
                 f"GROUP by c.id " \
                 f"HAVING count(*) > 100 " \
                 f"ORDER BY c.id"

#забираем способ приготовления рецепта с категориями из доступных с id категорий
sql = f"SELECT r.manual, (SELECT group_concat(rc.cat_id , ', ') " \
      f"from recipe_categories rc " \
      f"WHERE rc.recipe_id = r.id AND " \
      f"rc.cat_id in ({categories_sql})" \
      f") AS categories from recipe r"

loaded_data = pd.read_sql(sql, conn)
loaded_categories = pd.read_sql(categories_sql, conn)
#pandas data frame
conn.close()

In [3]:
#print(loaded_categories.id)

ids_cat = []
for i in range(len(loaded_categories)):
    ids_cat.append(str(loaded_categories.id[i]))

print(ids_cat)

['1', '2', '4', '5', '7', '8', '14', '16', '17', '20', '25', '27', '30', '33', '37', '43', '51', '62', '67', '68', '71', '90', '99', '103', '105', '139', '140', '150', '158', '257']


In [4]:
for i in range(len(ids_cat)):
    val = []
    for j in range(len(loaded_data)):
        spl = str(loaded_data.categories[j]).split(', ')
        #print(spl)
        #print(int(str(ids_cat[i]) in spl))
        val.append(int(str(ids_cat[i]) in spl))
    loaded_data.insert(loc=len(loaded_data.columns) , column=ids_cat[i], value=val)
#print(val)

In [5]:
#делим dataset на тренировочный и тестовый
x_train,x_test,y_train,y_test = train_test_split(loaded_data["manual"], loaded_data[ids_cat],test_size=0.3,random_state=42)

In [6]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [7]:
example = loaded_data.iloc[1]["manual"]
print(example)
print("------------------------")
print(tokenize_sentence(example))

Положите весь творог в кастрюльку и разомните его вилкой так, чтобы в нем не осталось крупных комков. Разбейте в него яйца, всыпьте сахар и тщательно все перемешайте. Лучше не использовать слишком сухой или слишком влажный творог, иначе сырники будут разваливаться в процессе приготовления.

Всыпьте в творог 5 столовых ложек (с горкой) муки и тщательно перемешайте. Можно добавить немного больше муки, сырники получатся тогда более плотными. Или муки можно добавить чуть меньше, и тогда сырники будут нежнее. В итоге у вас должна получиться однородная масса, из которой можно будет лепить сырники.

Поставьте сковороду на средний огонь и налейте в нее подсолнечное масло.

Насыпьте на тарелку немного муки. Слепите несколько небольших шариков из получившейся творожной массы и положите их на тарелку. Лучше лепить разом 4–5 шариков — столько, сколько поместится одновременно на сковороду. Затем по очереди обкатывайте творожные шарики в муке, плющите их в небольшие лепешки (они не должны быть слишк

In [8]:
#hyperparams
# Определение гиперпараметров для каждого компонента модели
tfidf_params = {
    'vectorizer__max_features': [500, 1000, 2000],
    'vectorizer__ngram_range': [(1, 1), (1, 2)],
    'vectorizer__use_idf': [True, False],
    # Другие гиперпараметры...
}
normalizer_params = {
    'normalizer__norm': ['l1', 'l2'],  # Тип нормализации (l2 - евклидова норма)
    # Другие гиперпараметры...
}

random_forest_params = {
    'model__estimator__n_estimators': [50, 100, 200],
    'model__estimator__max_depth': [None, 5, 10],
    'model__estimator__min_samples_split': [2, 5, 10],
    # Другие гиперпараметры...
}

In [9]:
tfidf = TfidfVectorizer()

Xfeatures = tfidf.fit_transform(x_train).toarray()

In [10]:
'''binary_rel_clf = BinaryRelevance(MultinomialNB()) #our model
binary_rel_clf.fit(Xfeatures, y_train)
'''
#старая версия

'binary_rel_clf = BinaryRelevance(MultinomialNB()) #our model\nbinary_rel_clf.fit(Xfeatures, y_train)\n'

In [27]:
from imblearn.over_sampling import

sampler = MultilabelOverSampler()
X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)

In [28]:
X_train_resampled, y_train_resampled = random_oversampling(x_train, y_train, ids_cat, ratio=2.0)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 30 and the array at index 1 has size 0

In [ ]:
model_pipeline = Pipeline([
    ("vectorizer",  TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("normalizer", Normalizer()),
    ("model", RandomForestClassifier())
]
)
model_pipeline.fit(x_train, y_train)
accuracy_score(y_test,model_pipeline.predict(x_test))

In [ ]:
test_text = "Выпечка"

model_pipeline.predict_proba([test_text])

In [ ]:
accuracy_score(y_test,model_pipeline.predict(x_test))

In [ ]:
precision_score(average='weighted', y_true=y_test, y_pred=model_pipeline.predict(x_test))

In [ ]:
recall_score(average='weighted', y_true=y_test, y_pred=model_pipeline.predict(x_test))

In [ ]:
y_score = model_pipeline.predict_proba(x_test)
print(y_score)

In [ ]:
print(y_score[:, 0])

In [ ]:
y_test['1']

In [ ]:
# precision recall curve
precision = dict()
recall = dict()

for i in range(len(ids_cat)):
    precision[i], recall[i], _ = precision_recall_curve(y_true=y_test[ids_cat[i]],
                                                        probas_pred=y_score[:, i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.show()

In [ ]:
#оценка гиперпараметров

grid_search = RandomizedSearchCV(model_pipeline, param_distributions={
    **tfidf_params,
    **normalizer_params,
    **random_forest_params
}, scoring='f1', cv=5, n_iter=2)

grid_search.fit(x_train, y_train)

In [ ]:
best_params = grid_search.best_params_

best_model = grid_search.best_estimator_

y_pred = best_model.predict_prob(x_test)